In [1]:
from sklearn import tree
import pandas as pd
import os

In [2]:
input_data_file = "output_data_npi.txt"
df = pd.read_csv(os.path.join("..", "Data", input_data_file),sep='\t')

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
#df = df.dropna()
df.head()

C:\Users\gleix\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,npi,nppes_provider_gender,nppes_entity_code,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,...,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,year,leie_code,leie
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012,None,Not
1,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,115.0,112.0,115.0,135.25,199.0,108.115652,2012,None,Not
2,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,93.0,88.0,93.0,198.59,291.0,158.870000,2012,None,Not
3,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,111.0,83.0,111.0,38.75,58.0,30.720721,2012,None,Not
4,1003000126,M,I,CUMBERLAND,2.15022e+08,MD,US,Internal Medicine,Y,F,...,N,544.0,295.0,544.0,70.95,105.0,56.655662,2012,None,Not


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56819227 entries, 0 to 56819226
Data columns (total 22 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   npi                               int64  
 1   nppes_provider_gender             object 
 2   nppes_entity_code                 object 
 3   nppes_provider_city               object 
 4   nppes_provider_zip                object 
 5   nppes_provider_state              object 
 6   nppes_provider_country            object 
 7   provider_type                     object 
 8   medicare_participation_indicator  object 
 9   place_of_service                  object 
 10  hcpcs_code                        object 
 11  hcpcs_description                 object 
 12  hcpcs_drug_indicator              object 
 13  line_srvc_cnt                     float64
 14  bene_unique_cnt                   float64
 15  bene_day_srvc_cnt                 float64
 16  average_Medicare_allowed_amt      

In [4]:
df.columns

Index(['npi', 'nppes_provider_gender', 'nppes_entity_code',
       'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',
       'nppes_provider_country', 'provider_type',
       'medicare_participation_indicator', 'place_of_service', 'hcpcs_code',
       'hcpcs_description', 'hcpcs_drug_indicator', 'line_srvc_cnt',
       'bene_unique_cnt', 'bene_day_srvc_cnt', 'average_Medicare_allowed_amt',
       'average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'year',
       'leie_code', 'leie'],
      dtype='object')

In [5]:
df['nppes_provider_gender'] = df['nppes_provider_gender'].fillna('O')

In [7]:
data = df.drop(['npi', 'nppes_provider_city', 'nppes_provider_zip', 'nppes_provider_state',              
                'nppes_provider_country', 'provider_type', 'hcpcs_code', 'hcpcs_description', 'year', 'leie_code'], axis=1)
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56819221 entries, 1 to 56819226
Data columns (total 12 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   nppes_provider_gender             object 
 1   nppes_entity_code                 object 
 2   medicare_participation_indicator  object 
 3   place_of_service                  object 
 4   hcpcs_drug_indicator              object 
 5   line_srvc_cnt                     float64
 6   bene_unique_cnt                   float64
 7   bene_day_srvc_cnt                 float64
 8   average_Medicare_allowed_amt      float64
 9   average_submitted_chrg_amt        float64
 10  average_Medicare_payment_amt      float64
 11  leie                              object 
dtypes: float64(6), object(6)
memory usage: 5.5+ GB


In [8]:
disposition = data["leie"]
disposition_names = ["On_List", "Not"]
data1 = data.drop("leie", axis=1)
feature_names = data1.columns

In [9]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56819221 entries, 1 to 56819226
Data columns (total 11 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   nppes_provider_gender             object 
 1   nppes_entity_code                 object 
 2   medicare_participation_indicator  object 
 3   place_of_service                  object 
 4   hcpcs_drug_indicator              object 
 5   line_srvc_cnt                     float64
 6   bene_unique_cnt                   float64
 7   bene_day_srvc_cnt                 float64
 8   average_Medicare_allowed_amt      float64
 9   average_submitted_chrg_amt        float64
 10  average_Medicare_payment_amt      float64
dtypes: float64(6), object(5)
memory usage: 5.1+ GB


In [10]:
disposition.shape

(56819221,)

In [12]:
test_df = data1.loc[(data["nppes_provider_gender"] == "O"), :]
test_df

,nppes_provider_gender,nppes_entity_code,medicare_participation_indicator,place_of_service,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt
120,O,O,Y,O,Y,353.0,353.0,353.0,30.920000,30.920000,30.920000
121,O,O,Y,O,N,421.0,421.0,421.0,21.490000,21.490333,21.490000
122,O,O,Y,O,Y,65.0,65.0,65.0,12.050000,12.453846,12.050000
225,O,O,Y,O,Y,94.0,94.0,94.0,29.590000,29.590000,29.590000
226,O,O,Y,O,Y,11.0,11.0,11.0,60.590000,60.590000,60.590000
...,...,...,...,...,...,...,...,...,...,...,...
56819204,O,O,Y,F,N,36.0,33.0,36.0,288.477778,1500.000000,219.702222
56819205,O,O,Y,F,N,48.0,40.0,48.0,296.720000,1500.000000,226.542500
56819206,O,O,Y,F,N,22.0,22.0,22.0,2803.287273,7500.000000,2197.769091
56819207,O,O,Y,F,N,66.0,42.0,49.0,325.651667,1500.000000,248.943030


In [16]:
new_data = pd.get_dummies(data1)
new_data

,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,nppes_provider_gender_F,nppes_provider_gender_M,nppes_provider_gender_O,nppes_entity_code_I,nppes_entity_code_O,medicare_participation_indicator_N,medicare_participation_indicator_Y,place_of_service_F,place_of_service_O,hcpcs_drug_indicator_N,hcpcs_drug_indicator_Y
1,115.0,112.0,115.0,135.25,199.0,108.115652,0,1,0,1,0,0,1,1,0,1,0
2,93.0,88.0,93.0,198.59,291.0,158.870000,0,1,0,1,0,0,1,1,0,1,0
3,111.0,83.0,111.0,38.75,58.0,30.720721,0,1,0,1,0,0,1,1,0,1,0
4,544.0,295.0,544.0,70.95,105.0,56.655662,0,1,0,1,0,0,1,1,0,1,0
5,75.0,55.0,75.0,101.74,150.0,81.390000,0,1,0,1,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56819222,248.0,175.0,248.0,116.86,345.0,85.637177,0,1,0,1,0,0,1,0,1,1,0
56819223,41.0,41.0,41.0,203.40,496.0,225.564634,1,0,0,1,0,0,1,1,0,1,0
56819224,101.0,57.0,101.0,72.48,176.0,83.200000,1,0,0,1,0,0,1,1,0,1,0
56819225,102.0,55.0,102.0,104.76,254.0,119.450784,1,0,0,1,0,0,1,1,0,1,0


In [17]:
new_data.columns

Index(['line_srvc_cnt', 'bene_unique_cnt', 'bene_day_srvc_cnt',
       'average_Medicare_allowed_amt', 'average_submitted_chrg_amt',
       'average_Medicare_payment_amt', 'nppes_provider_gender_F',
       'nppes_provider_gender_M', 'nppes_provider_gender_O',
       'nppes_entity_code_I', 'nppes_entity_code_O',
       'medicare_participation_indicator_N',
       'medicare_participation_indicator_Y', 'place_of_service_F',
       'place_of_service_O', 'hcpcs_drug_indicator_N',
       'hcpcs_drug_indicator_Y'],
      dtype='object')

In [18]:
new_data = new_data.drop(['nppes_entity_code_O', 'medicare_participation_indicator_Y', 'place_of_service_O', 'hcpcs_drug_indicator_Y'], axis=1)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56819221 entries, 1 to 56819226
Data columns (total 13 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   line_srvc_cnt                       float64
 1   bene_unique_cnt                     float64
 2   bene_day_srvc_cnt                   float64
 3   average_Medicare_allowed_amt        float64
 4   average_submitted_chrg_amt          float64
 5   average_Medicare_payment_amt        float64
 6   nppes_provider_gender_F             uint8  
 7   nppes_provider_gender_M             uint8  
 8   nppes_provider_gender_O             uint8  
 9   nppes_entity_code_I                 uint8  
 10  medicare_participation_indicator_N  uint8  
 11  place_of_service_F                  uint8  
 12  hcpcs_drug_indicator_N              uint8  
dtypes: float64(6), uint8(7)
memory usage: 3.3 GB


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_data, disposition, random_state=42)

In [20]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42614415 entries, 46790733 to 56755042
Data columns (total 13 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   line_srvc_cnt                       float64
 1   bene_unique_cnt                     float64
 2   bene_day_srvc_cnt                   float64
 3   average_Medicare_allowed_amt        float64
 4   average_submitted_chrg_amt          float64
 5   average_Medicare_payment_amt        float64
 6   nppes_provider_gender_F             uint8  
 7   nppes_provider_gender_M             uint8  
 8   nppes_provider_gender_O             uint8  
 9   nppes_entity_code_I                 uint8  
 10  medicare_participation_indicator_N  uint8  
 11  place_of_service_F                  uint8  
 12  hcpcs_drug_indicator_N              uint8  
dtypes: float64(6), uint8(7)
memory usage: 2.5 GB


In [21]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9978402380152183

In [22]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9989650685831261

In [23]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2977642751322113, 'line_srvc_cnt'),
 (0.2647143221321886, 'place_of_service'),
 (0.19945025244943348, 'hcpcs_drug_indicator'),
 (0.08877768297304331, 'nppes_entity_code'),
 (0.07656018816014105, 'nppes_provider_gender'),
 (0.07126911601195372, 'medicare_participation_indicator'),
 (0.00017409214377972387, 'bene_day_srvc_cnt'),
 (0.0001347165431930609, 'bene_unique_cnt'),
 (7.158573166483138e-05, 'average_Medicare_allowed_amt'),
 (6.887808436955663e-05, 'average_submitted_chrg_amt'),
 (6.414644192385288e-06, 'average_Medicare_payment_amt')]